# Testing + mini tutorial of pyvisa to control the oscilliscope

Notes
- To check what variables are available to call or change on the oscilliscope, open XStreamBrowser on the oscilliscope. The tree in the browser gives you the path and the information on particular variable tells you the type, whether you can read/write, and possible ingputs (Range/Helpstring)
    - Example: app.acquisition.horizontal.maximize has the options "FixedSampleRate" and "SetMaximumMemory" (still not sure what this means, but it didn't work until I used one of these)

In [1]:
import pyvisa

In [2]:
python -m visa info

SyntaxError: invalid syntax (<ipython-input-2-ae66e3687d1f>, line 1)

In [2]:
# Before running, connect the laptop to the oscilloscope through the USBTMC
rm = pyvisa.ResourceManager()
rm.list_resources() # use this to grab the instrument in the next step

('USB0::0x05FF::0x1023::4206N21552::INSTR', 'ASRL3::INSTR', 'ASRL10::INSTR')

In [3]:
# Grabs the instrument and prints
scope = rm.open_resource('USB0::0x05FF::0x1023::4206N21552::INSTR')

# Message for “what are you?”
scope.query('*IDN?')

'*IDN LECROY,WAVERUNNER8054,LCRY4206N21552,8.5.1\n'

In [4]:
# Change the vertical scale and horizontal scale of channel 1
# (good way to check if we can send commands to the scope)
scope.write(r"""vbs 'app.Acquisition.C1.VerScale = 1.0' """)
scope.write(r"""vbs 'app.Acquisition.Horizontal.HorScale = 500e-6' """)

(53, <StatusCode.success: 0>)

In [6]:
# Print the vertical offset for channel 1 
# (good way to check that we can recieve info from the scope)
(scope.query(r"""vbs? 'return=app.Acquisition.C1.VerScale' """))

'VBS 2\n'

In [19]:
# Waits 5 seconds after oscilliscope is idle before 
# setting up new acquisition
scope.timeout = 5000
scope.clear()
r = scope.query(r"""vbs? 'return=app.WaitUntilIdle(5)' """)

In [40]:
# Set up aquisition by stopping trigger 
trigger_level = .85
scope.write(r"""vbs 'app.acquisition.triggermode = "stopped" ' """)
scope.write(r"""vbs 'app.acquisition.trigger.edge.level = %f ' """ % trigger_level)
scope.write(r"""vbs 'app.acquisition.triggermode = "single" ' """)
scope.write(r"""vbs 'app.acquisition.horizontal.maximize = "FixedSampleRate" ' """)

(65, <StatusCode.success: 0>)

In [46]:
# Set up Measurements, clearing previous measurements
scope.write(r"""vbs 'app.clearsweeps ' """)
scope.write(r"""vbs 'app.measure.clearall ' """)
scope.write(r"""vbs 'app.measure.clearsweeps ' """)


# Setup averaging waveform
scope.write(r"""vbs 'app.math.f1.operator1 = "average" ' """)
scope.write(r"""vbs 'app.math.f1.source1 = "C1" ' """)
# Sweep
num_sweeps = 100
for i in range(0,num_sweeps):
    r = scope.query(r"""vbs? 'return=app.acquisition.acquire( 0.1 , True ) ' """)
    r = scope.query(r"""vbs? 'return=app.WaitUntilIdle(5)' """)
    if r==0:
        print("Time out from WaitUntilIdle, return = {0}".format(r))


# Setup measurement of peak to peak voltage based on average waveform
scope.write(r"""vbs 'app.measure.showmeasure = true ' """)
scope.write(r"""vbs 'app.measure.statson = true ' """)
scope.write(r"""vbs 'app.measure.p1.view = true ' """)
scope.write(r"""vbs 'app.measure.p1.paramengine = "pkpk" ' """ ) 
scope.write(r"""vbs 'app.measure.p1.source1 = "F1" ' """)
r = scope.query(r"""vbs? 'return=app.acquisition.acquire( 0.1 , True ) ' """)
r = scope.query(r"""vbs? 'return=app.WaitUntilIdle(.3)' """)

# Read back Measurement Results
result = scope.query(r"""vbs? 'return=app.measure.p1.out.result.value' """)
print("Result = {0}".format(result))

Result = 0



In [33]:
# Aquire the data!
for i in range(0,1000):
    r = scope.query(r"""vbs? 'return=app.acquisition.acquire( 0.1 , True ) ' """)
    r = scope.query(r"""vbs? 'return=app.WaitUntilIdle(5)' """)
    if r==0:
        print("Time out from WaitUntilIdle, return = {0}".format(r))


In [32]:
# Read back Measurement Results
result = scope.query(r"""vbs? 'return=app.measure.p1.out.result.value' """)
print("Result = {0}".format(result))

Result = 7.09852630883688



In [6]:
# Make sure to close the connection (in teh reverse order of setting it up)
scope.close()
rm.close()